In [1]:

import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision.utils import make_grid

In [2]:
torch.manual_seed(0)
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
def train(model, dataset, n_iter=100):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.NLLLoss()
    model.train()
    
    for epoch in range(n_iter):
        for images, targets in dataset:
            optimizer.zero_grad()
            images = images.view(images.size(0), -1)
            out = model(images.to(device))
            loss = criterion(out, targets.to(device))
            loss.backward()
            optimizer.step()
        
        if epoch % 10 == 0:
            print('epoch: %3d loss: %.4f'%(epoch, loss))

def accuracy(model, trainset):
    model.eval()
    correct = 0
    for images, targets in trainset:
        images = images.view(images.size(0), -1)
        correct += (model(images.to(device)).argmax(dim=1)==targets.to(device)).sum()
    return correct / len(trainset)

In [4]:
data_dir = './trainset'
dataset = datasets.ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((100,100)),transforms.ToTensor()
]))

batch_size = 128
val_size = 500
train_size = len(dataset) - val_size 

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

Length of Train Data : 2500
Length of Validation Data : 500


In [5]:
def show_batch(dl):
    """Plot images grid of single batch"""
    for images, labels in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
show_batch(train_dl)

In [5]:
hidden_size=300

model = nn.Sequential(
    nn.Linear(100 * 100 * 3, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, 10), 
    nn.LogSoftmax(dim=-1)
).to(device)
train(model, train_dl)


epoch:   0 loss: 1.2318
epoch:  10 loss: 1.0683
epoch:  20 loss: 1.0015
epoch:  30 loss: 1.3194
epoch:  40 loss: 1.1256
epoch:  50 loss: 1.0612
epoch:  60 loss: 0.8555
epoch:  70 loss: 0.8731
epoch:  80 loss: 0.7595
epoch:  90 loss: 0.7440


In [6]:
accuracy(model, val_dl)
torch.save(model, "neural_network_model.pth")

In [12]:
another_model = torch.load("neural_network_model.pth")
another_model.eval()
print(accuracy(another_model, val_dl))

tensor(161.5000)


In [9]:
from PIL import Image

In [17]:
img = Image.open("../bombs/hcb/hcb-0.png")

transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor()
])
input_tensor = transform(img).unsqueeze(0).to(device)

# Reshape the input tensor
batch_size = input_tensor.size(0)
input_tensor = input_tensor.view(batch_size, -1)

# Perform inference
with torch.no_grad():
    output = model(input_tensor)

# Interpret the output
predicted_class = output.argmax(dim=1)

print(f'Predicted Bomb: {predicted_class.item()}')

Predicted Bomb: 1
